# Future Extension : Improve VQE results by classical post-processing

This is an idea which is inspired from this [paper](https://arxiv.org/pdf/1801.04849.pdf). The thesis is that if VQE (for any hamiltonian fragment) is producing noisy results, it should hopefully be in the neighborhood of the sample we obtained (in z basis).  

Thus, if we can use a technique like Simulated Annealing, give it the samples from VQE and a small initial temperature. It will search the locality of the initial temperature and try to get us a sample and a lower energy (eigenvalue) or just a smaller optimization cutoff (in terms of time).

The eventual goal is not to replace VQE with SA, but to complement it by plugging in results from VQE into SA to try to get a better value. 

In [125]:
#(Requires D-wave's Ocean SDK)
from  additional_dwave import *

#The Ising cost function in h J of the allZ Hamiltonian from S4 (of the qwc fragment) 
h = {0:0.15542669077992818, 1: 0.1062290449085607, 2: 0.16326768673564326 }
J = {(0,1):0.15660062488237944, (0,2): 0.1062290449085607, (0,3): -0.04919764587136754,
    (1,2):0.15542669077992818, (2,3):0.04919764587136754,
    (0,1,3):0.04919764587136754, (1,2,3):-0.04919764587136754}
offset = -0.3276081896748086 #for the hamiltonian term that didn't have any operators associated with it

#Make a 'poly' dictionary that is acceptable for the next function
poly = {}
for key in h:
    poly[(key,)] = h[key]
for key in J:
    poly[key] = J[key]

#This is the unoptimized initial value
init_state = {0:-1,1:-1,2:-1,3:1}
print("Our initial state (its 1 bit off from the actuall answer:", init_state)
print("Our initial state energy:",float(poly_energies(init_state,poly)))

Our initial state (its 1 bit off from the actuall answer: {0: -1, 1: -1, 2: -1, 3: 1}
Our initial state energy: -0.006667061853263816


Now the first step is to reduce the higher order polynomial to a binary quadratic problem

In [111]:

bqm = make_quadratic(poly, 1.0, vartype=dimod.SPIN)
#Make your init_state compatible with the quadratic expanded model
init_state_quad = expand_initial_state(bqm, init_state)

A little bit of pre-processing here

In [112]:
#Unfortunately, in order to supply an initial state, the SA sampler doesn't take string keys, so we do the following dance
bqm_lin = {}
bqm_quad = {}
int2str_dict = {}
var_name_ctr = 4 #instead of string names, we'll assign these, (or else we run into issues later on)

for key in bqm.linear:
    
    if isinstance(key, int):
        bqm_lin[key] = bqm.linear[key]
    else:
        bqm_lin[var_name_ctr] = bqm.linear[key]
        int2str_dict[key] = var_name_ctr
        var_name_ctr += 1

key0 = None
key1 = None
for key in bqm.quadratic:
    if isinstance(key[0], int):
        key0 = key[0]
    else:
        key0 = int2str_dict[key[0]]
    
    if isinstance(key[1], int):
        key1 = key[1]
    else:
        key1 = int2str_dict[key[1]]
    
    bqm_quad[(key0,key1)] = bqm.quadratic[key]

#use the list form to store the initial state
init_list = []
for key in init_state_quad: #lets 'read' a sample out of the dict the same way it presents its variables
    init_list.append(init_state_quad[key])

Get the default beta (inverse temperature) range

In [113]:
b1,b2 = default_ising_beta_range(bqm_lin, bqm_quad)
print("Start beta: ",b1)
print("Stop beta: ",b2)

Start beta:  0.17328679513998632
Stop beta:  93.60549888969886


Let's start with the default SA process

In [117]:
newsampler = neal.SimulatedAnnealingSampler()

#default settings
response = newsampler.sample_ising(bqm.linear,bqm.quadratic,num_reads=4,seed=4,num_sweep=2)
print("########")
print("For default settings")
for i in range(0,len(response.record)):
    answer_sample = response.record[i][0][0:4]
    print(answer_sample)
    print(float(poly_energies(answer_sample,poly)))
print("Best energy (eigenvalue): ",-0.41826360336075774+offset)


########
For default settings
[-1  1 -1  1]
-0.41826360336075774
[-1 -1  1 -1]
-0.40023374324442523
[-1  1 -1 -1]
-0.41826360336075774
[-1 -1  1 -1]
-0.40023374324442523
Best energy (eigenvalue):  -0.7458717930355663


This is the smallest eigenvalue of the allz qwc fragment in S4 for H<sub>2</sub>

Let's try a localized search : smaller number of steps and our initial state

In [118]:
newsampler = neal.SimulatedAnnealingSampler()

#default settings
response = newsampler.sample_ising(bqm.linear,bqm.quadratic,num_reads=4,seed=4,beta_range=(100,100.1),num_sweeps=2,init_state=init_list)
print("########")
print("For default settings")
for i in range(0,len(response.record)):
    answer_sample = response.record[i][0][0:4]
    print(answer_sample)
    print(float(poly_energies(answer_sample,poly)))
print("Best energy (eigenvalue): ",-0.41826360336075774+offset)

########
For default settings
[-1  1 -1  1]
-0.41826360336075774
[-1  1 -1  1]
-0.41826360336075774
[-1 -1  1 -1]
-0.40023374324442523
[-1  1 -1  1]
-0.41826360336075774
Best energy (eigenvalue):  -0.7458717930355663


Now, obviously, number of steps = 2 is ridiculous (but our problem size here is very small). The general idea is that we want a speedier SA which can help improve results as well.

We hope that this idea can be expanded upon/tested further in the future.